In [ ]:
!python3 -m pip install pyppeteer
!apt install libxrender1 libxtst6 libxi6

!pip install ffmpy

!pip install webp
!pip uninstall Pillow
!pip install Pillow

!pip install piexif

  Using cached pyppeteer-0.2.2-py3-none-any.whl (145 kB)
  Using cached pyee-7.0.2-py2.py3-none-any.whl (12 kB)
     |████████████████████████████████| 78 kB 5.4 MB/s eta 0:00:011
E: Could not open lock file /var/lib/dpkg/lock-frontend - open (13: Permission denied)
E: Unable to acquire the dpkg frontend lock (/var/lib/dpkg/lock-frontend), are you root?
  Using cached ffmpy-0.2.3.tar.gz (4.6 kB)
  Created wheel for ffmpy: filename=ffmpy-0.2.3-py3-none-any.whl size=4621 sha256=62b40df89070dd6fe5bfbf78e14d80374d86bb58423dfc83a4f24383dcd9f445
  Stored in directory: /home/jovyan/.cache/pip/wheels/89/ba/63/44887ceb692de6586cf0788c901dace9b3a371e914a9b4ad79
Successfully built ffmpy
  Using cached webp-0.1.0a16.tar.gz (10 kB)
  Created wheel for webp: filename=webp-0.1.0a16-cp38-cp38-linux_x86_64.whl size=445229 sha256=d98938c754f566524bb0884a70bd6e35a5f783ccbb46d62ccedc67bd033b878d
  Stored in directory: /home/jovyan/.cache/pip/wheels/f5/ea/92/8d98d79a5f24a9c8f841747c7300d66510b74a09dc2f4176

In [2]:
import os
from pathlib import Path

In [3]:
images_dir = Path('../images')
recycle_dir = Path('../recycle')

In [4]:
recycle_dir.mkdir(parents=True, exist_ok=True)

# Convert .svg to .png

Tried svglib, cairosvg and phantomjs, all are error prone. Improper font, color, etc.

Finally settled with puppeteer, the output images are rendered just like chrome does

In [5]:
from pyppeteer import launch

In [6]:
async def svg2png (in_svg_path, out_png_path):
    browser = await launch(args=['--no-sandbox'])
    page = await browser.newPage()
    await page.goto(f"file://{os.path.abspath(in_svg_path)}")
    dimensions = await page.evaluate('''() => {
        return {
            width: document.documentElement.width.baseVal.value,
            height: document.documentElement.height.baseVal.value
        }
    }''')
    await page.setViewport({'width': int(dimensions['width']), 'height': int(dimensions['height'])})
    await page.screenshot({'path': out_png_path})
    await browser.close()

In [7]:
for root, dirs, files in os.walk(images_dir):
    for name in files:
        try:
            if name.endswith('.svg'):
                basename, ext = os.path.splitext(name)
                f_in = os.path.join(root, name)
                f_out = os.path.join(root, f"{basename}.png")
                print(f_in)
                await svg2png(f_in, f_out)
                f_mv = recycle_dir/name
                !mv {f_in} {f_mv}
        except Exception as inst:
            print(f"Error converting image: {name} {inst}")

# Convert Imgur .gifv format, which downloaded as .mp4, to .gif

In [8]:
import ffmpy

In [9]:
for root, dirs, files in os.walk(images_dir):
    for name in files:
        try:
            if name.endswith('.mp4'):
                basename, ext = os.path.splitext(name)
                f_in = os.path.join(root, name)
                f_out = os.path.join(root, f"{basename}.gif")
                print(f_in)
                !rm {f_out}
                ffmpy.FFmpeg(inputs={f_in: None}, outputs={f_out: None}).run()
                f_mv = recycle_dir/name
                !mv {f_in} {f_mv}
        except Exception as inst:
            print(f"Error converting video: {name} {inst}")

# Convert .bmp to .png

In [10]:
import imageio

In [11]:
for root, dirs, files in os.walk(images_dir):
    for name in files:
        basename, ext = os.path.splitext(name)
        if ext == '.bmp':
            print(os.path.join(root, name))
            f_in = os.path.join(root, name)
            f_out = os.path.join(root, f"{basename}.png")
            im = imageio.imread(os.path.join(root, name))
            imageio.imwrite(f_out, im)

            f_mv = recycle_dir/name
            !mv {f_in} {f_mv}

# Convert .webp to .png

In [12]:
from PIL import Image

In [13]:
for root, dirs, files in os.walk(images_dir):
    for name in files:
        basename, ext = os.path.splitext(name)
        if ext == '.webp':
            print(os.path.join(root, name))
            f_in = os.path.join(root, name)
            f_out = os.path.join(root, f"{basename}.png")
            im = Image.open(os.path.join(root, name))
            im.save(f_out, 'png')

            f_mv = recycle_dir/name
            !mv {f_in} {f_mv}

# Correct extension

In [14]:
import os

In [15]:
# id: '2w19o1' downloaded from 'http://i.imgur.com/GE7LJR6.pngh' as .pngh

for root, dirs, files in os.walk(images_dir):
    for name in files:
        basename, ext = os.path.splitext(name)
        if ext == '.jpeg':
            print(os.path.join(root, name))
            os.rename(os.path.join(root, name), os.path.join(root, basename) + '.jpg')
        if ext == '.pngh':
            print(os.path.join(root, name))
            os.rename(os.path.join(root, name), os.path.join(root, basename) + '.png')

# Delete invalid file

In [16]:
# id: '4f927p' downloaded from 'http://imgur.com/(null)' as '4f927p(null'

for root, dirs, files in os.walk(images_dir):
    for name in files:
        basename, ext = os.path.splitext(name)
        if ext == '':
            print(os.path.join(root, name))
            os.remove(os.path.join(root, name))

# Repair corrupted files

In [17]:
import piexif

In [18]:
# import warnings
# warnings.filterwarnings("error")

In [19]:
# Error digesting image: 63027680638.jpg Corrupt EXIF data.  Expecting to read 2 bytes but only got 0.
# im = Image.open('images/preview/tumblr/wtf-viz/63027680638.jpg')
# piexif.remove('images/preview/tumblr/wtf-viz/63027680638.jpg')

In [20]:
# Error digesting image: 63027680638.jpg Corrupt EXIF data.  Expecting to read 2 bytes but only got 0.
# im = Image.open('images/thumbnail/tumblr/wtf-viz/63027680638.jpg')
# piexif.remove('images/thumbnail/tumblr/wtf-viz/63027680638.jpg')

In [21]:
# Error digesting image: 2jtymg.jpg Image size (135364608 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
# 39744830 bytes ~37.9MB
# huge image, nothing to do
im = Image.open('images/archive/reddit/dataisugly/2jtymg.jpg')

/opt/conda/lib/python3.7/site-packages/PIL/Image.py:2800: DecompressionBombWarning: Image size (135364608 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  DecompressionBombWarning,


In [22]:
# Error digesting image: 9g6pzu.png Image size (100000000 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
# 2208964 bytes ~
# huge image, nothing to do
im = Image.open('images/external_link/reddit/dataisugly/9g6pzu.png')

/opt/conda/lib/python3.7/site-packages/PIL/Image.py:2800: DecompressionBombWarning: Image size (100000000 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  DecompressionBombWarning,


In [23]:
# Error digesting image: anqyqb.png Palette images with Transparency expressed in bytes should be converted to RGBA images
# Happens when converting transparent .png to grayscale, nothing to do
im = Image.open('images/preview/reddit/dataisbeautiful/anqyqb.png')
imL = im.convert('L')

/opt/conda/lib/python3.7/site-packages/PIL/Image.py:960: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  "Palette images with Transparency expressed in bytes should be "
